In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_selection import RFE
import warnings
warnings.filterwarnings("ignore")

In [2]:
election_data_path = '../../datasets/Clean Datasets/Electoral Data/'
census_data_path = '../../datasets/Clean Datasets/Census Data/'
master_data_path = '../../datasets/Clean Datasets/'

In [3]:
votes = pd.read_csv(election_data_path + 'Final_Vote_Count.csv')
data = pd.read_csv(master_data_path + 'CED to All Demographics and Results.csv')
ages = pd.read_csv(census_data_path + 'CED to Age (Individual years).csv')
master = pd.read_csv(master_data_path + 'CED to All Demographics and Results Norm.csv')

In [4]:
No_NA_master = master.dropna().reset_index()
No_NA_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,RELP - Inadequately described,RELP - Not stated,SEXP - Male,SEXP - Female,MDCP - Married in a registered marriage,MDCP - Married in a de facto marriage,MDCP - Not married,MDCP - Not applicable,PreferencePercent,PartyNm
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,0.005066,0.089304,0.495587,0.504413,0.302321,0.073786,0.364082,0.259811,58.18,Australian Labor Party
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,0.005405,0.070051,0.488707,0.511293,0.393949,0.060754,0.305368,0.239929,60.13,Liberal
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,0.005846,0.089031,0.483762,0.516238,0.323363,0.081842,0.301819,0.292976,60.98,Australian Labor Party
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,0.002856,0.069741,0.486358,0.513642,0.385881,0.044820,0.335227,0.234073,56.26,Liberal
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,0.004301,0.092769,0.505114,0.494886,0.359438,0.078180,0.273848,0.288534,68.94,Liberal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,0.002762,0.083012,0.493321,0.506679,0.358736,0.039473,0.313827,0.287964,55.47,Labor
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,0.003842,0.073615,0.485514,0.514486,0.366147,0.067582,0.294500,0.271771,60.91,Labor
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,0.004758,0.104341,0.494793,0.505207,0.350901,0.080181,0.284154,0.284764,63.15,Liberal National Party of Queensland
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,0.006079,0.085782,0.489373,0.510627,0.304067,0.093583,0.361030,0.241320,58.17,Australian Labor Party


In [5]:
set(No_NA_master['PartyNm'])

{'Australian Labor Party',
 'Australian Labor Party (Northern Territory) Branch',
 'Centre Alliance',
 'Independent',
 "Katter's Australian Party (KAP)",
 'Labor',
 'Liberal',
 'Liberal National Party of Queensland',
 'The Greens (VIC)',
 'The Nationals'}

# 1) Adjusted Model partitioning into LNP, ALP, Other 

In [6]:
adjusted_master = No_NA_master.replace({ 'PartyNm' : {'Liberal' : 'LNP', 'Liberal National Party of Queensland' : 'LNP',
                                                      'The Nationals' : 'LNP', 'Labor' : 'ALP',
                                                      'Australian Labor Party' : 'ALP',
                                                     'Australian Labor Party (Northern Territory) Branch' : 'ALP',
                                                      'Centre Alliance' : 'Other', 'Independent' : 'Other', 
                                                      'Katter\'s Australian Party (KAP)' : 'Other',
                                                      'The Greens (VIC)' : 'Other'}})

adjusted_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,RELP - Inadequately described,RELP - Not stated,SEXP - Male,SEXP - Female,MDCP - Married in a registered marriage,MDCP - Married in a de facto marriage,MDCP - Not married,MDCP - Not applicable,PreferencePercent,PartyNm
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,0.005066,0.089304,0.495587,0.504413,0.302321,0.073786,0.364082,0.259811,58.18,ALP
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,0.005405,0.070051,0.488707,0.511293,0.393949,0.060754,0.305368,0.239929,60.13,LNP
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,0.005846,0.089031,0.483762,0.516238,0.323363,0.081842,0.301819,0.292976,60.98,ALP
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,0.002856,0.069741,0.486358,0.513642,0.385881,0.044820,0.335227,0.234073,56.26,LNP
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,0.004301,0.092769,0.505114,0.494886,0.359438,0.078180,0.273848,0.288534,68.94,LNP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,0.002762,0.083012,0.493321,0.506679,0.358736,0.039473,0.313827,0.287964,55.47,ALP
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,0.003842,0.073615,0.485514,0.514486,0.366147,0.067582,0.294500,0.271771,60.91,ALP
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,0.004758,0.104341,0.494793,0.505207,0.350901,0.080181,0.284154,0.284764,63.15,LNP
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,0.006079,0.085782,0.489373,0.510627,0.304067,0.093583,0.361030,0.241320,58.17,ALP


In [7]:
len(adjusted_master[adjusted_master['PartyNm'] == 'LNP'])

77

In [8]:
len(adjusted_master[adjusted_master['PartyNm'] == 'ALP'])

68

In [9]:
len(adjusted_master[adjusted_master['PartyNm'] == 'Other'])

6

# 2) Strength of victory model

Definitions by the AEC https://www.aec.gov.au/footer/glossary.htm#:~:text=Marginal%20seat,per%20cent%20of%20the%20vote.

- Marginal seat 56% or less
- Fairly safe seat 56-60%
- Safe seat 60% or greater

In [10]:
strength = No_NA_master[['LGA', 'PartyNm','PreferencePercent']]

In [11]:
Safe = strength[strength['PreferencePercent'] >= 60]
set(Safe['PartyNm'])

{'Australian Labor Party',
 'Independent',
 "Katter's Australian Party (KAP)",
 'Labor',
 'Liberal',
 'Liberal National Party of Queensland',
 'The Greens (VIC)',
 'The Nationals'}

In [12]:
strength[(strength['PreferencePercent'] < 60) & (strength['PreferencePercent'] > 56)]

,LGA,PartyNm,PreferencePercent
0,Adelaide,Australian Labor Party,58.18
3,Banks,Liberal,56.26
5,Barton,Labor,59.41
7,Bean,Australian Labor Party,57.52
8,Bendigo,Australian Labor Party,59.04
9,Bennelong,Liberal,56.91
13,Bonner,Liberal National Party of Queensland,57.41
18,Brand,Australian Labor Party,56.66
36,Cowper,The Nationals,56.79
52,Flynn,Liberal National Party of Queensland,58.66


In [13]:
strength[strength['PreferencePercent'] <= 56]

,LGA,PartyNm,PreferencePercent
6,Bass,Liberal,50.41
11,Blair,Australian Labor Party,51.21
14,Boothby,Liberal,51.38
16,Braddon,Liberal,53.09
19,Brisbane,Liberal National Party of Queensland,54.92
21,Burt,Australian Labor Party,54.99
27,Casey,Liberal,54.64
29,Chisholm,Liberal,50.57
33,Corangamite,Australian Labor Party,51.07
35,Cowan,Australian Labor Party,50.83


In [14]:
conditions = [(adjusted_master['PreferencePercent'] <= 56),
              ((adjusted_master['PreferencePercent'] > 56) & (adjusted_master['PreferencePercent'] < 60 )
              & (adjusted_master['PartyNm'] == 'LNP')),
              ((adjusted_master['PreferencePercent'] > 56) & (adjusted_master['PreferencePercent'] < 60 )
              & (adjusted_master['PartyNm'] == 'ALP')),
              (adjusted_master['PreferencePercent'] >= 60) & (adjusted_master['PartyNm'] == 'LNP'),
              (adjusted_master['PreferencePercent'] > 56) & (adjusted_master['PartyNm'] == 'ALP'),
             (adjusted_master['PreferencePercent'] > 56) & (adjusted_master['PartyNm'] == 'Other')]

strengths = ['Marginal', 'Fairly Safe LNP', 'Fairly Safe ALP', 'Safe LNP', 'Safe ALP', 'Other']

adjusted_master['Strength'] = np.select(conditions, strengths)
adjusted_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,RELP - Not stated,SEXP - Male,SEXP - Female,MDCP - Married in a registered marriage,MDCP - Married in a de facto marriage,MDCP - Not married,MDCP - Not applicable,PreferencePercent,PartyNm,Strength
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,0.089304,0.495587,0.504413,0.302321,0.073786,0.364082,0.259811,58.18,ALP,Fairly Safe ALP
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,0.070051,0.488707,0.511293,0.393949,0.060754,0.305368,0.239929,60.13,LNP,Safe LNP
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,0.089031,0.483762,0.516238,0.323363,0.081842,0.301819,0.292976,60.98,ALP,Safe ALP
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,0.069741,0.486358,0.513642,0.385881,0.044820,0.335227,0.234073,56.26,LNP,Fairly Safe LNP
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,0.092769,0.505114,0.494886,0.359438,0.078180,0.273848,0.288534,68.94,LNP,Safe LNP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,0.083012,0.493321,0.506679,0.358736,0.039473,0.313827,0.287964,55.47,ALP,Marginal
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,0.073615,0.485514,0.514486,0.366147,0.067582,0.294500,0.271771,60.91,ALP,Safe ALP
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,0.104341,0.494793,0.505207,0.350901,0.080181,0.284154,0.284764,63.15,LNP,Safe LNP
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,0.085782,0.489373,0.510627,0.304067,0.093583,0.361030,0.241320,58.17,ALP,Fairly Safe ALP


In [15]:
set(adjusted_master['Strength'])

{'Fairly Safe ALP',
 'Fairly Safe LNP',
 'Marginal',
 'Other',
 'Safe ALP',
 'Safe LNP'}

# 3) Correct Model (ALP, LNP, Other)

Forgot non-ordinal categorical vars can't be regressed on a single output, need dummy vars & regress on those. 

created dummy variables:
- is_ALP
- is_LNP
- is_Other

In [16]:
adjusted_master['is_ALP'] = np.where(adjusted_master['PartyNm'] == 'ALP', 1, 0)
adjusted_master['is_LNP'] = np.where(adjusted_master['PartyNm'] == 'LNP', 1, 0)
adjusted_master['is_Other'] = np.where(adjusted_master['PartyNm'] == 'Other', 1, 0)
adjusted_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,MDCP - Married in a registered marriage,MDCP - Married in a de facto marriage,MDCP - Not married,MDCP - Not applicable,PreferencePercent,PartyNm,Strength,is_ALP,is_LNP,is_Other
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,0.302321,0.073786,0.364082,0.259811,58.18,ALP,Fairly Safe ALP,1,0,0
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,0.393949,0.060754,0.305368,0.239929,60.13,LNP,Safe LNP,0,1,0
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,0.323363,0.081842,0.301819,0.292976,60.98,ALP,Safe ALP,1,0,0
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,0.385881,0.044820,0.335227,0.234073,56.26,LNP,Fairly Safe LNP,0,1,0
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,0.359438,0.078180,0.273848,0.288534,68.94,LNP,Safe LNP,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,0.358736,0.039473,0.313827,0.287964,55.47,ALP,Marginal,1,0,0
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,0.366147,0.067582,0.294500,0.271771,60.91,ALP,Safe ALP,1,0,0
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,0.350901,0.080181,0.284154,0.284764,63.15,LNP,Safe LNP,0,1,0
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,0.304067,0.093583,0.361030,0.241320,58.17,ALP,Fairly Safe ALP,1,0,0


# 3.1 ALP model

In [42]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 146)
(31, 146)


In [43]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_train = train['is_ALP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_test = test['is_ALP']

In [44]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.7416666666666667
Accuracy score on testing set:  0.6129032258064516


In [45]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.5483870967741935
Accuracy on test set using 2 features:  0.5483870967741935
Accuracy on test set using 3 features:  0.6129032258064516
Accuracy on test set using 4 features:  0.5483870967741935
Accuracy on test set using 5 features:  0.5806451612903226
Accuracy on test set using 6 features:  0.6129032258064516
Accuracy on test set using 7 features:  0.6129032258064516
Accuracy on test set using 8 features:  0.6129032258064516
Accuracy on test set using 9 features:  0.5806451612903226
Accuracy on test set using 10 features:  0.5483870967741935
Accuracy on test set using 11 features:  0.5483870967741935
Accuracy on test set using 12 features:  0.5483870967741935
Accuracy on test set using 13 features:  0.5483870967741935
Accuracy on test set using 14 features:  0.5483870967741935
Accuracy on test set using 15 features:  0.6129032258064516
Accuracy on test set using 16 features:  0.6129032258064516
Accuracy on test set using 17 features:  0.612903

In [46]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=3, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.6129032258064516


In [47]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 26.000
Column: 1, Selected False, Rank: 123.000
Column: 2, Selected False, Rank: 56.000
Column: 3, Selected False, Rank: 87.000
Column: 4, Selected False, Rank: 15.000
Column: 5, Selected False, Rank: 7.000
Column: 6, Selected False, Rank: 6.000
Column: 7, Selected False, Rank: 22.000
Column: 8, Selected False, Rank: 82.000
Column: 9, Selected False, Rank: 68.000
Column: 10, Selected False, Rank: 39.000
Column: 11, Selected False, Rank: 31.000
Column: 12, Selected False, Rank: 25.000
Column: 13, Selected False, Rank: 18.000
Column: 14, Selected False, Rank: 21.000
Column: 15, Selected False, Rank: 32.000
Column: 16, Selected False, Rank: 53.000
Column: 17, Selected False, Rank: 61.000
Column: 18, Selected False, Rank: 79.000
Column: 19, Selected False, Rank: 107.000
Column: 20, Selected False, Rank: 134.000
Column: 21, Selected False, Rank: 71.000
Column: 22, Selected False, Rank: 112.000
Column: 23, Selected False, Rank: 75.000
Column: 24, Selected Fal

In [48]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_ALP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [86, 124, 126]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,1,0,62.21
134,1,0,54.45
107,0,1,71.83
78,1,0,52.98
128,0,0,53.18
65,1,0,52.80


In [49]:
best_cols = X_test.iloc[:,relevant_cols]
best_cols

,MSTP - Never married,RELP - Christianity,RELP - Islam
3,0.260373,0.555461,0.048503
70,0.331367,0.584734,0.005375
92,0.257813,0.561709,0.001849
124,0.265419,0.502974,0.011615
56,0.255628,0.492437,0.002484
134,0.250313,0.619925,0.001979
107,0.591751,0.260394,0.035241
78,0.260744,0.632586,0.002672
128,0.319403,0.457105,0.027582
65,0.233375,0.527592,0.048839


In [50]:
list_name = best_cols.columns
list_name

Index(['MSTP - Never married', 'RELP - Christianity', 'RELP - Islam'], dtype='object')

In [51]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,MSTP - Never married,RELP - Christianity,RELP - Islam
0,0.67747,-1.245888,1.028113
1,0.67747,-1.245888,1.028113


# 3.1 Interpretation (is_ALP) (61%)

The main determinants of whether an electorate is votes ALP or not, is:
- The proprtion of people who are Christian (significantly negatively affects it)
- The proprtion of people who are Muslim (significantly positively affects it)
- The proprtion of people who have never married (Slightly positively affects it)

# Number of seats ALP Win

In [53]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(adjusted_master['is_ALP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(adjusted_master.drop(['index', 'LGA','PartyNm', 'PreferencePercent',
                                                                'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)))
result.insert(2, "Pref Percent", adjusted_master['PreferencePercent'].values)
result.insert(3, "Electorate", adjusted_master['LGA'].values)
display(result)

print('Number of seats the ALP win:',result['Predicted'].sum())

Columns that provide the best are: [86, 124, 126]


,Actual,Predicted,Pref Percent,Electorate
0,1,1,58.18,Adelaide
1,0,0,60.13,Aston
2,1,0,60.98,Ballarat
3,0,0,56.26,Banks
4,0,0,68.94,Barker
...,...,...,...,...
146,1,0,55.47,Werriwa
147,1,0,60.91,Whitlam
148,0,0,63.15,Wide Bay
149,1,1,58.17,Wills


Number of seats the ALP win: 16


In [54]:
ALP_Wins = result[result['Predicted'] == 1]
ALP_Wins['Electorate']

0        Adelaide
12       Blaxland
19       Brisbane
20          Bruce
23        Calwell
24       Canberra
32         Cooper
64      Grayndler
97      Macnamara
107     Melbourne
122    Parramatta
139          Swan
140        Sydney
144        Watson
145     Wentworth
149         Wills
Name: Electorate, dtype: object

# 3.2 is_LNP model

In [55]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 146)
(31, 146)


In [56]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_train = train['is_LNP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_test = test['is_LNP']

In [57]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.725
Accuracy score on testing set:  0.6451612903225806


In [58]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.6129032258064516
Accuracy on test set using 2 features:  0.6129032258064516
Accuracy on test set using 3 features:  0.5806451612903226
Accuracy on test set using 4 features:  0.5483870967741935
Accuracy on test set using 5 features:  0.5483870967741935
Accuracy on test set using 6 features:  0.5483870967741935
Accuracy on test set using 7 features:  0.5806451612903226
Accuracy on test set using 8 features:  0.5806451612903226
Accuracy on test set using 9 features:  0.5806451612903226
Accuracy on test set using 10 features:  0.5806451612903226
Accuracy on test set using 11 features:  0.5806451612903226
Accuracy on test set using 12 features:  0.5806451612903226
Accuracy on test set using 13 features:  0.5806451612903226
Accuracy on test set using 14 features:  0.5483870967741935
Accuracy on test set using 15 features:  0.5483870967741935
Accuracy on test set using 16 features:  0.5806451612903226
Accuracy on test set using 17 features:  0.548387

In [59]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=47, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.6451612903225806


In [60]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected True, Rank: 1.000
Column: 1, Selected False, Rank: 66.000
Column: 2, Selected False, Rank: 11.000
Column: 3, Selected False, Rank: 33.000
Column: 4, Selected True, Rank: 1.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected True, Rank: 1.000
Column: 7, Selected True, Rank: 1.000
Column: 8, Selected False, Rank: 31.000
Column: 9, Selected False, Rank: 22.000
Column: 10, Selected True, Rank: 1.000
Column: 11, Selected True, Rank: 1.000
Column: 12, Selected True, Rank: 1.000
Column: 13, Selected True, Rank: 1.000
Column: 14, Selected True, Rank: 1.000
Column: 15, Selected True, Rank: 1.000
Column: 16, Selected False, Rank: 6.000
Column: 17, Selected False, Rank: 15.000
Column: 18, Selected False, Rank: 32.000
Column: 19, Selected False, Rank: 63.000
Column: 20, Selected False, Rank: 89.000
Column: 21, Selected False, Rank: 17.000
Column: 22, Selected False, Rank: 80.000
Column: 23, Selected False, Rank: 40.000
Column: 24, Selected False, Rank: 49.000
Column: 

In [61]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_LNP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [0, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 25, 28, 31, 38, 39, 46, 47, 51, 61, 66, 69, 71, 72, 77, 79, 80, 83, 86, 87, 90, 91, 92, 97, 106, 118, 119, 122, 123, 124, 125, 126, 128, 129, 134, 136, 137]


,Actual,Predicted,Pref Percent
3,1,1,56.26
70,1,0,58.36
92,1,1,53.28
124,1,1,57.52
56,0,1,62.21
134,0,1,54.45
107,0,0,71.83
78,0,1,52.98
128,1,0,53.18
65,0,0,52.80


In [62]:
best_cols = X_test.iloc[:,relevant_cols]
best_cols

,AGE5P - 0-4 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,AGE5P - 50-54 years,AGE5P - 55-59 years,AGE5P - 60-64 years,AGE5P - 65-69 years,AGE5P - 70-74 years,...,RLHP - Not applicable,RELP - Buddhism,RELP - Christianity,RELP - Hinduism,RELP - Islam,RELP - Other Religions,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Married in a registered marriage,MDCP - Not married,MDCP - Not applicable
3,0.060613,0.068598,0.074162,0.073559,0.066018,0.068046,0.065087,0.054882,0.046442,0.035024,...,0.017688,0.042913,0.555461,0.031552,0.048503,0.005776,0.242016,0.385881,0.335227,0.234073
70,0.070830,0.086721,0.078522,0.074623,0.064649,0.061693,0.058431,0.048320,0.042489,0.028672,...,0.053760,0.008369,0.584734,0.004756,0.005375,0.004806,0.290817,0.280871,0.293952,0.324590
92,0.064828,0.062505,0.060643,0.060699,0.058862,0.063701,0.061023,0.059491,0.059192,0.046097,...,0.027490,0.008668,0.561709,0.003076,0.001849,0.005013,0.318683,0.339021,0.282468,0.292313
124,0.079176,0.064734,0.076436,0.078890,0.070544,0.063458,0.051827,0.042782,0.039294,0.027305,...,0.035168,0.012099,0.502974,0.013953,0.011615,0.010976,0.342444,0.340102,0.244735,0.325028
56,0.059398,0.048807,0.050483,0.059065,0.058893,0.067738,0.073947,0.068101,0.065627,0.047807,...,0.035667,0.007118,0.492437,0.002534,0.002484,0.003877,0.400382,0.351234,0.279460,0.278461
134,0.058333,0.055385,0.050907,0.056568,0.055316,0.066985,0.068452,0.063455,0.062632,0.052789,...,0.024605,0.008112,0.619925,0.002810,0.001979,0.002962,0.293455,0.366205,0.304889,0.253644
107,0.037696,0.172723,0.182719,0.135079,0.078709,0.039943,0.035943,0.031038,0.026816,0.018287,...,0.062582,0.060602,0.260394,0.026891,0.035241,0.004693,0.472482,0.181893,0.453533,0.239190
78,0.065358,0.058623,0.059331,0.061920,0.058527,0.064810,0.066347,0.061193,0.058119,0.045005,...,0.042456,0.006760,0.632586,0.002985,0.002672,0.003029,0.260657,0.340778,0.281919,0.297467
128,0.058885,0.087894,0.114632,0.106807,0.079441,0.059232,0.054545,0.045155,0.037870,0.027664,...,0.035074,0.049320,0.457105,0.071167,0.027582,0.006292,0.285639,0.359918,0.336725,0.243467
65,0.080754,0.060545,0.070088,0.088415,0.091308,0.058728,0.051643,0.044769,0.036943,0.024190,...,0.017887,0.020750,0.527592,0.121368,0.048839,0.049709,0.156307,0.400018,0.274307,0.281916


In [63]:
list_name = best_cols.columns
list_name

Index(['AGE5P - 0-4 years', 'AGE5P - 20-24 years', 'AGE5P - 25-29 years',
       'AGE5P - 30-34 years', 'AGE5P - 35-39 years', 'AGE5P - 50-54 years',
       'AGE5P - 55-59 years', 'AGE5P - 60-64 years', 'AGE5P - 65-69 years',
       'AGE5P - 70-74 years', 'AGE5P - 75-79 years',
       'HEAP - Certificate III & IV Level',
       'HEAP - Secondary Education - Years 9 and below',
       'HEAP - Not applicable', 'INCP - $2,000-$2,999 ($104,000-$155,999)',
       'INCP - $3,000 or more ($156,000 or more)', 'INCP - Nil income',
       'INCP - Not applicable', 'INDP - Agriculture, Forestry and Fishing',
       'INDP - Mining', 'INDP - Public Administration and Safety',
       'INDP - Transport, Postal and Warehousing',
       'LFHRP - Employed, worked full-time',
       'LFHRP - Employed, worked part-time', 'LFHRP - Not in the labour force',
       'LFHRP - Not applicable', 'GNGP - National Government',
       'GNGP - Private sector', 'MSTP - Never married', 'MSTP - Widowed',
       'MSTP - M

In [64]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,AGE5P - 0-4 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,AGE5P - 50-54 years,AGE5P - 55-59 years,AGE5P - 60-64 years,AGE5P - 65-69 years,AGE5P - 70-74 years,...,RLHP - Not applicable,RELP - Buddhism,RELP - Christianity,RELP - Hinduism,RELP - Islam,RELP - Other Religions,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Married in a registered marriage,MDCP - Not married,MDCP - Not applicable
0,-0.19152,-0.250676,-0.393,-0.401863,-0.228084,0.119506,0.156255,0.192222,0.246282,0.227102,...,0.203385,-0.395329,1.152395,-0.303826,-0.865314,-0.152802,0.493993,0.414232,-0.561112,0.14731
1,-0.19152,-0.250676,-0.393,-0.401863,-0.228084,0.119506,0.156255,0.192222,0.246282,0.227102,...,0.203385,-0.395329,1.152395,-0.303826,-0.865314,-0.152802,0.493993,0.414232,-0.561112,0.14731


In [65]:
coefficient_DF.iloc[0]

AGE5P - 0-4 years                                                                 -0.191520
AGE5P - 20-24 years                                                               -0.250676
AGE5P - 25-29 years                                                               -0.393000
AGE5P - 30-34 years                                                               -0.401863
AGE5P - 35-39 years                                                               -0.228084
AGE5P - 50-54 years                                                                0.119506
AGE5P - 55-59 years                                                                0.156255
AGE5P - 60-64 years                                                                0.192222
AGE5P - 65-69 years                                                                0.246282
AGE5P - 70-74 years                                                                0.227102
AGE5P - 75-79 years                                                             

# 3.2 Interpretation (is_LNP) (65%)

## Significant Positive Influence
- Christian (1.152395) 
- Married (0.519304)
- Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation (0.493993)
- registered marriage (0.414232)
- Husband, Wife or Partner in a registered marriage (0.392106)

## Significant Negative Influence
- Islam (-0.865314)
- Never married (-0.618718)
- Not married (-0.561112)
- 30-34 years (-0.401863)
- Buddhism (-0.395329)
- 25-29 years (-0.393000)

# Number of Seats LNP Win

In [69]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(adjusted_master['is_LNP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(adjusted_master.drop(['index', 'LGA','PartyNm', 'PreferencePercent',
                                                                'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)))
result.insert(2, "Pref Percent", adjusted_master['PreferencePercent'].values)
result.insert(3, "Electorate", adjusted_master['LGA'].values)
display(result)

print('Number of seats the LNP win:',result['Predicted'].sum())

Columns that provide the best are: [0, 4, 5, 6, 7, 10, 11, 12, 13, 14, 15, 25, 28, 31, 38, 39, 46, 47, 51, 61, 66, 69, 71, 72, 77, 79, 80, 83, 86, 87, 90, 91, 92, 97, 106, 118, 119, 122, 123, 124, 125, 126, 128, 129, 134, 136, 137]


,Actual,Predicted,Pref Percent,Electorate
0,0,0,58.18,Adelaide
1,1,1,60.13,Aston
2,0,1,60.98,Ballarat
3,1,1,56.26,Banks
4,1,1,68.94,Barker
...,...,...,...,...
146,0,0,55.47,Werriwa
147,0,1,60.91,Whitlam
148,1,1,63.15,Wide Bay
149,0,0,58.17,Wills


Number of seats the LNP win: 98


In [70]:
LNP_Wins = result[result['Predicted'] == 1]
LNP_Wins['Electorate']

1          Aston
2       Ballarat
3          Banks
4         Barker
6           Bass
         ...    
142       Wannon
143    Warringah
147      Whitlam
148     Wide Bay
150       Wright
Name: Electorate, Length: 98, dtype: object

# 3.3 is_Other Model

In [71]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 146)
(31, 146)


In [72]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_train = train['is_Other']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)
y_test = test['is_Other']

In [73]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.9666666666666667
Accuracy score on testing set:  0.9354838709677419


In [74]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.9354838709677419
Accuracy on test set using 2 features:  0.9354838709677419
Accuracy on test set using 3 features:  0.9354838709677419
Accuracy on test set using 4 features:  0.9354838709677419
Accuracy on test set using 5 features:  0.9354838709677419
Accuracy on test set using 6 features:  0.9354838709677419
Accuracy on test set using 7 features:  0.9354838709677419
Accuracy on test set using 8 features:  0.9354838709677419
Accuracy on test set using 9 features:  0.9354838709677419
Accuracy on test set using 10 features:  0.9354838709677419
Accuracy on test set using 11 features:  0.9354838709677419
Accuracy on test set using 12 features:  0.9354838709677419
Accuracy on test set using 13 features:  0.9354838709677419
Accuracy on test set using 14 features:  0.9354838709677419
Accuracy on test set using 15 features:  0.9354838709677419
Accuracy on test set using 16 features:  0.9354838709677419
Accuracy on test set using 17 features:  0.935483

In [75]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=1, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.9354838709677419


In [76]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 47.000
Column: 1, Selected False, Rank: 93.000
Column: 2, Selected False, Rank: 74.000
Column: 3, Selected False, Rank: 66.000
Column: 4, Selected False, Rank: 18.000
Column: 5, Selected False, Rank: 21.000
Column: 6, Selected False, Rank: 23.000
Column: 7, Selected False, Rank: 42.000
Column: 8, Selected False, Rank: 108.000
Column: 9, Selected False, Rank: 80.000
Column: 10, Selected False, Rank: 53.000
Column: 11, Selected False, Rank: 33.000
Column: 12, Selected False, Rank: 35.000
Column: 13, Selected False, Rank: 34.000
Column: 14, Selected False, Rank: 36.000
Column: 15, Selected False, Rank: 63.000
Column: 16, Selected False, Rank: 81.000
Column: 17, Selected False, Rank: 85.000
Column: 18, Selected False, Rank: 109.000
Column: 19, Selected False, Rank: 130.000
Column: 20, Selected False, Rank: 133.000
Column: 21, Selected False, Rank: 55.000
Column: 22, Selected False, Rank: 76.000
Column: 23, Selected False, Rank: 41.000
Column: 24, Selected F

In [77]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_Other'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [129]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,0,0,62.21
134,0,0,54.45
107,1,0,71.83
78,0,0,52.98
128,0,0,53.18
65,0,0,52.80


In [78]:
best_cols = X_test.iloc[:,relevant_cols]
best_cols

,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation
3,0.242016
70,0.290817
92,0.318683
124,0.342444
56,0.400382
134,0.293455
107,0.472482
78,0.260657
128,0.285639
65,0.156307


In [79]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation
0,0.1457
1,0.1457


# 3.3 Interpretation is_Other (Caution) (94%)

Though The model says it a high accuracy at 93%, this is a misguided interpretation as for all outputs it is outputing 0, i.e. not other. Hence this model can't predict whether an electorate is not ALP/LNP. The highest coefficient is (0.1457) it is very weak which explains why the model can't accurately predict a seat that is 'Other'. I would contend that the reasons why this Logistic regression model can't predict 'Other' electorates is due to there not being enough of them in order to correctly train the model, the total dataset has only 7 data points, also within the 'Other' category I have a group of party's like the Greens with Katters' Australian Party, Greens' have left-wing policies, KAP have Right-wing, I would doubt the composition of these electorates are similar.   

# Number of seats Other wins

In [82]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(adjusted_master['is_LNP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(adjusted_master.drop(['index', 'LGA','PartyNm', 'PreferencePercent',
                                                                'Strength', 'is_ALP', 'is_LNP', 'is_Other'], axis=1)))
result.insert(2, "Pref Percent", adjusted_master['PreferencePercent'].values)
result.insert(3, "Electorate", adjusted_master['LGA'].values)
display(result)

print('Number of seats the Other win:',result['Predicted'].sum())

Columns that provide the best are: [129]


,Actual,Predicted,Pref Percent,Electorate
0,0,0,58.18,Adelaide
1,1,0,60.13,Aston
2,0,0,60.98,Ballarat
3,1,0,56.26,Banks
4,1,0,68.94,Barker
...,...,...,...,...
146,0,0,55.47,Werriwa
147,0,0,60.91,Whitlam
148,1,0,63.15,Wide Bay
149,0,0,58.17,Wills


Number of seats the Other win: 0


In [81]:
Other_Wins = result[result['Predicted'] == 1]
Other_Wins['Electorate']

Series([], Name: Electorate, dtype: object)

# 4 More Specific model (Safe seats)

In [58]:
adjusted_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,MDCP - Married in a registered marriage,MDCP - Married in a de facto marriage,MDCP - Not married,MDCP - Not applicable,PreferencePercent,PartyNm,Strength,is_ALP,is_LNP,is_Other
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,0.302321,0.073786,0.364082,0.259811,58.18,ALP,Fairly Safe ALP,1,0,0
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,0.393949,0.060754,0.305368,0.239929,60.13,LNP,Safe LNP,0,1,0
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,0.323363,0.081842,0.301819,0.292976,60.98,ALP,Safe ALP,1,0,0
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,0.385881,0.044820,0.335227,0.234073,56.26,LNP,Fairly Safe LNP,0,1,0
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,0.359438,0.078180,0.273848,0.288534,68.94,LNP,Safe LNP,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,0.358736,0.039473,0.313827,0.287964,55.47,ALP,Marginal,1,0,0
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,0.366147,0.067582,0.294500,0.271771,60.91,ALP,Safe ALP,1,0,0
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,0.350901,0.080181,0.284154,0.284764,63.15,LNP,Safe LNP,0,1,0
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,0.304067,0.093583,0.361030,0.241320,58.17,ALP,Fairly Safe ALP,1,0,0


In [59]:
adjusted_master['is_Marginal'] = np.where(adjusted_master['Strength'] == 'Marginal', 1, 0)
adjusted_master['is_Fairly_Safe_ALP'] = np.where(adjusted_master['Strength'] == 'Fairly Safe ALP', 1, 0)
adjusted_master['is_Fairly_Safe_LNP'] = np.where(adjusted_master['Strength'] == 'Fairly Safe LNP', 1, 0)
adjusted_master['is_Safe_ALP'] = np.where(adjusted_master['Strength'] == 'Safe ALP', 1, 0)
adjusted_master['is_Safe_LNP'] = np.where(adjusted_master['Strength'] == 'Safe LNP', 1, 0)
adjusted_master['is_Strength_Other'] = np.where(adjusted_master['Strength'] == 'Other', 1, 0)
adjusted_master

,index,LGA,AGE5P - 0-4 years,AGE5P - 5-9 years,AGE5P - 10-14 years,AGE5P - 15-19 years,AGE5P - 20-24 years,AGE5P - 25-29 years,AGE5P - 30-34 years,AGE5P - 35-39 years,...,Strength,is_ALP,is_LNP,is_Other,is_Marginal,is_Fairly_Safe_ALP,is_Fairly_Safe_LNP,is_Safe_ALP,is_Safe_LNP,is_Strength_Other
0,0,Adelaide,0.053945,0.050723,0.045462,0.057361,0.096777,0.092626,0.088211,0.070221,...,Fairly Safe ALP,1,0,0,0,1,0,0,0,0
1,1,Aston,0.058553,0.058845,0.058346,0.064261,0.066246,0.064391,0.067479,0.066700,...,Safe LNP,0,1,0,0,0,0,0,1,0
2,2,Ballarat,0.062608,0.066055,0.061419,0.062910,0.065860,0.060216,0.059006,0.059537,...,Safe ALP,1,0,0,0,0,0,1,0,0
3,3,Banks,0.060613,0.059368,0.055980,0.060472,0.068598,0.074162,0.073559,0.066018,...,Fairly Safe LNP,0,1,0,0,0,1,0,0,0
4,4,Barker,0.055697,0.063267,0.062771,0.059757,0.050169,0.051558,0.054241,0.054597,...,Safe LNP,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,164,Werriwa,0.072655,0.078559,0.077535,0.077236,0.072380,0.065587,0.071789,0.070232,...,Marginal,1,0,0,1,0,0,0,0,0
147,165,Whitlam,0.060113,0.065839,0.064873,0.066358,0.055673,0.052804,0.056448,0.056678,...,Safe ALP,1,0,0,0,0,0,1,0,0
148,166,Wide Bay,0.048858,0.060062,0.064301,0.056468,0.042020,0.040191,0.044199,0.049902,...,Safe LNP,0,1,0,0,0,0,0,1,0
149,167,Wills,0.062568,0.052921,0.043203,0.043616,0.080062,0.112030,0.108720,0.087122,...,Fairly Safe ALP,1,0,0,0,1,0,0,0,0


# 4.1 Marginal Model

In [145]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [146]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Marginal']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Marginal']

In [147]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.7
Accuracy score on testing set:  0.6129032258064516


In [148]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.6129032258064516
Accuracy on test set using 2 features:  0.6129032258064516
Accuracy on test set using 3 features:  0.6129032258064516
Accuracy on test set using 4 features:  0.6129032258064516
Accuracy on test set using 5 features:  0.6129032258064516
Accuracy on test set using 6 features:  0.6129032258064516
Accuracy on test set using 7 features:  0.6129032258064516
Accuracy on test set using 8 features:  0.6129032258064516
Accuracy on test set using 9 features:  0.6129032258064516
Accuracy on test set using 10 features:  0.6129032258064516
Accuracy on test set using 11 features:  0.6129032258064516
Accuracy on test set using 12 features:  0.6129032258064516
Accuracy on test set using 13 features:  0.6129032258064516
Accuracy on test set using 14 features:  0.6129032258064516
Accuracy on test set using 15 features:  0.6129032258064516
Accuracy on test set using 16 features:  0.6129032258064516
Accuracy on test set using 17 features:  0.612903

In [149]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=10, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.6129032258064516


In [150]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 94.000
Column: 1, Selected False, Rank: 67.000
Column: 2, Selected False, Rank: 81.000
Column: 3, Selected False, Rank: 95.000
Column: 4, Selected False, Rank: 45.000
Column: 5, Selected False, Rank: 16.000
Column: 6, Selected False, Rank: 13.000
Column: 7, Selected False, Rank: 71.000
Column: 8, Selected False, Rank: 87.000
Column: 9, Selected False, Rank: 86.000
Column: 10, Selected False, Rank: 77.000
Column: 11, Selected False, Rank: 68.000
Column: 12, Selected False, Rank: 84.000
Column: 13, Selected False, Rank: 62.000
Column: 14, Selected False, Rank: 50.000
Column: 15, Selected False, Rank: 41.000
Column: 16, Selected False, Rank: 46.000
Column: 17, Selected False, Rank: 70.000
Column: 18, Selected False, Rank: 90.000
Column: 19, Selected False, Rank: 123.000
Column: 20, Selected False, Rank: 128.000
Column: 21, Selected False, Rank: 44.000
Column: 22, Selected False, Rank: 42.000
Column: 23, Selected False, Rank: 11.000
Column: 24, Selected Fal

In [151]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_Marginal'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [62, 77, 85, 86, 90, 92, 124, 126, 129, 134]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,1,0,53.28
124,0,0,57.52
56,0,0,62.21
134,1,0,54.45
107,0,0,71.83
78,1,0,52.98
128,1,0,53.18
65,1,0,52.80


In [77]:
best_cols = X_test.iloc[:,relevant_cols]
best_cols

,INDP - Not applicable,LFHRP - Not in the labour force,GNGP - Not applicable,MSTP - Never married,MSTP - Married,"RLHP - Husband, Wife or Partner in a registered marriage",RELP - Christianity,RELP - Islam,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Married in a registered marriage
3,0.533226,0.289307,0.533239,0.260373,0.429529,0.390934,0.555461,0.048503,0.242016,0.385881
70,0.542880,0.227649,0.542897,0.331367,0.325917,0.284470,0.584734,0.005375,0.290817,0.280871
92,0.589752,0.291475,0.589748,0.257813,0.381666,0.344130,0.561709,0.001849,0.318683,0.339021
124,0.535209,0.207218,0.535174,0.265419,0.391204,0.343378,0.502974,0.011615,0.342444,0.340102
56,0.559708,0.308677,0.559928,0.255628,0.398361,0.353628,0.492437,0.002484,0.400382,0.351234
134,0.570357,0.318004,0.570325,0.250313,0.402176,0.370344,0.619925,0.001979,0.293455,0.366205
107,0.487229,0.260181,0.487512,0.591751,0.232039,0.183481,0.260394,0.035241,0.472482,0.181893
78,0.583925,0.305707,0.583959,0.260744,0.387295,0.344979,0.632586,0.002672,0.260657,0.340778
128,0.504883,0.267467,0.504869,0.319403,0.422374,0.363494,0.457105,0.027582,0.285639,0.359918
65,0.517665,0.225184,0.517650,0.233375,0.438427,0.404093,0.527592,0.048839,0.156307,0.400018


In [78]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,INDP - Not applicable,LFHRP - Not in the labour force,GNGP - Not applicable,MSTP - Never married,MSTP - Married,"RLHP - Husband, Wife or Partner in a registered marriage",RELP - Christianity,RELP - Islam,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Married in a registered marriage
0,-0.211056,-0.232745,-0.205949,0.218063,-0.169172,-0.20466,-0.637355,-0.205417,0.617044,-0.197312
1,-0.211056,-0.232745,-0.205949,0.218063,-0.169172,-0.20466,-0.637355,-0.205417,0.617044,-0.197312


# 4.1 Interpretation of Marginal seats (61% Accurate)


## Strongest determinants
- Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation (0.617044)
- Christianity (-0.637355)

## Weaker determinants
- Not in the labour force (-0.232745)
- Never married (0.218063)
- INDP - Not applicable (-0.211056)
- GNGP - Not applicable (-0.205949)
- Islam (-0.205417)
- Husband, Wife or Partner in a registered marriage (-0.20466)

# 4.2 Fairly Safe ALP Model

In [87]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [88]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Fairly_Safe_ALP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Fairly_Safe_ALP']

In [89]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.8833333333333333
Accuracy score on testing set:  0.8709677419354839


In [90]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.8709677419354839
Accuracy on test set using 2 features:  0.8709677419354839
Accuracy on test set using 3 features:  0.8709677419354839
Accuracy on test set using 4 features:  0.8709677419354839
Accuracy on test set using 5 features:  0.8709677419354839
Accuracy on test set using 6 features:  0.8709677419354839
Accuracy on test set using 7 features:  0.8709677419354839
Accuracy on test set using 8 features:  0.8709677419354839
Accuracy on test set using 9 features:  0.8709677419354839
Accuracy on test set using 10 features:  0.8709677419354839
Accuracy on test set using 11 features:  0.8709677419354839
Accuracy on test set using 12 features:  0.8709677419354839
Accuracy on test set using 13 features:  0.8709677419354839
Accuracy on test set using 14 features:  0.8709677419354839
Accuracy on test set using 15 features:  0.8709677419354839
Accuracy on test set using 16 features:  0.8709677419354839
Accuracy on test set using 17 features:  0.870967

In [91]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.8709677419354839


In [92]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 36.000
Column: 1, Selected False, Rank: 87.000
Column: 2, Selected False, Rank: 33.000
Column: 3, Selected False, Rank: 50.000
Column: 4, Selected False, Rank: 26.000
Column: 5, Selected False, Rank: 8.000
Column: 6, Selected False, Rank: 7.000
Column: 7, Selected False, Rank: 11.000
Column: 8, Selected False, Rank: 60.000
Column: 9, Selected False, Rank: 95.000
Column: 10, Selected False, Rank: 45.000
Column: 11, Selected False, Rank: 27.000
Column: 12, Selected False, Rank: 17.000
Column: 13, Selected False, Rank: 15.000
Column: 14, Selected False, Rank: 24.000
Column: 15, Selected False, Rank: 44.000
Column: 16, Selected False, Rank: 72.000
Column: 17, Selected False, Rank: 94.000
Column: 18, Selected False, Rank: 105.000
Column: 19, Selected False, Rank: 114.000
Column: 20, Selected False, Rank: 127.000
Column: 21, Selected False, Rank: 34.000
Column: 22, Selected False, Rank: 99.000
Column: 23, Selected False, Rank: 12.000
Column: 24, Selected Fals

In [95]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_ALP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [83, 86, 124, 126, 136]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,1,0,62.21
134,1,0,54.45
107,0,0,71.83
78,1,0,52.98
128,0,0,53.18
65,1,0,52.80


In [96]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,GNGP - Private sector,MSTP - Never married,RELP - Christianity,RELP - Islam,MDCP - Not married
0,0.189134,0.275993,-0.424978,0.198243,0.220149
1,0.189134,0.275993,-0.424978,0.198243,0.220149


# 4.2) Fairly Safe ALP Interpretation (87% Accurate)

## Strongest determinant
- Christianity (-0.424978)

## Weaker determinants
- Never married (0.275993)
- Not married (0.220149)
- Islam (0.198243)
- Private sector (0.189134)

# 4.3) Fairly Safe LNP

In [97]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [98]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Fairly_Safe_LNP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Fairly_Safe_LNP']

In [99]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.9
Accuracy score on testing set:  0.8387096774193549


In [100]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.8387096774193549
Accuracy on test set using 2 features:  0.8387096774193549
Accuracy on test set using 3 features:  0.8387096774193549
Accuracy on test set using 4 features:  0.8387096774193549
Accuracy on test set using 5 features:  0.8387096774193549
Accuracy on test set using 6 features:  0.8387096774193549
Accuracy on test set using 7 features:  0.8387096774193549
Accuracy on test set using 8 features:  0.8387096774193549
Accuracy on test set using 9 features:  0.8387096774193549
Accuracy on test set using 10 features:  0.8387096774193549
Accuracy on test set using 11 features:  0.8387096774193549
Accuracy on test set using 12 features:  0.8387096774193549
Accuracy on test set using 13 features:  0.8387096774193549
Accuracy on test set using 14 features:  0.8387096774193549
Accuracy on test set using 15 features:  0.8387096774193549
Accuracy on test set using 16 features:  0.8387096774193549
Accuracy on test set using 17 features:  0.838709

Accuracy on test set using 139 features:  0.8387096774193549
1 columns provide the highest accuracy at: 83.87 %


In [101]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.8387096774193549


In [102]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 40.000
Column: 1, Selected False, Rank: 108.000
Column: 2, Selected False, Rank: 85.000
Column: 3, Selected False, Rank: 99.000
Column: 4, Selected False, Rank: 31.000
Column: 5, Selected False, Rank: 9.000
Column: 6, Selected False, Rank: 12.000
Column: 7, Selected False, Rank: 48.000
Column: 8, Selected False, Rank: 93.000
Column: 9, Selected False, Rank: 84.000
Column: 10, Selected False, Rank: 70.000
Column: 11, Selected False, Rank: 71.000
Column: 12, Selected False, Rank: 46.000
Column: 13, Selected False, Rank: 34.000
Column: 14, Selected False, Rank: 38.000
Column: 15, Selected False, Rank: 56.000
Column: 16, Selected False, Rank: 69.000
Column: 17, Selected False, Rank: 58.000
Column: 18, Selected False, Rank: 75.000
Column: 19, Selected False, Rank: 111.000
Column: 20, Selected False, Rank: 129.000
Column: 21, Selected False, Rank: 5.000
Column: 22, Selected False, Rank: 54.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Selected False,

In [103]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_ALP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [23, 92, 124, 126, 134]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,1,0,62.21
134,1,0,54.45
107,0,0,71.83
78,1,0,52.98
128,0,0,53.18
65,1,0,52.80


In [104]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,HEAP - Bachelor Degree Level,"RLHP - Husband, Wife or Partner in a registered marriage",RELP - Christianity,RELP - Islam,MDCP - Married in a registered marriage
0,0.26231,0.180321,0.300811,-0.200075,0.182068
1,0.26231,0.180321,0.300811,-0.200075,0.182068


# 4.3 Fairly Safe LNP (84% Accurate)

## Determinants (All are fairly weak)

- Christianity (0.300811)
- Bachelor Degree Level (0.26231)
- Islam (-0.200075)
- Married in a registered marriage (0.182068)
- Husband, Wife or Partner in a registered marriage (0.180321)

# 4.4 Safe ALP

In [138]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [139]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Safe_ALP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Safe_ALP']

In [140]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.8166666666666667
Accuracy score on testing set:  0.9032258064516129


In [141]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.9032258064516129
Accuracy on test set using 2 features:  0.9032258064516129
Accuracy on test set using 3 features:  0.9032258064516129
Accuracy on test set using 4 features:  0.9032258064516129
Accuracy on test set using 5 features:  0.9032258064516129
Accuracy on test set using 6 features:  0.9032258064516129
Accuracy on test set using 7 features:  0.9032258064516129
Accuracy on test set using 8 features:  0.9032258064516129
Accuracy on test set using 9 features:  0.9032258064516129
Accuracy on test set using 10 features:  0.9032258064516129
Accuracy on test set using 11 features:  0.9032258064516129
Accuracy on test set using 12 features:  0.9032258064516129
Accuracy on test set using 13 features:  0.9032258064516129
Accuracy on test set using 14 features:  0.9032258064516129
Accuracy on test set using 15 features:  0.9032258064516129
Accuracy on test set using 16 features:  0.9032258064516129
Accuracy on test set using 17 features:  0.903225

In [142]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.9032258064516129


In [143]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 38.000
Column: 1, Selected False, Rank: 114.000
Column: 2, Selected False, Rank: 68.000
Column: 3, Selected False, Rank: 115.000
Column: 4, Selected False, Rank: 8.000
Column: 5, Selected False, Rank: 6.000
Column: 6, Selected False, Rank: 9.000
Column: 7, Selected False, Rank: 31.000
Column: 8, Selected False, Rank: 109.000
Column: 9, Selected False, Rank: 61.000
Column: 10, Selected False, Rank: 50.000
Column: 11, Selected False, Rank: 39.000
Column: 12, Selected False, Rank: 27.000
Column: 13, Selected False, Rank: 20.000
Column: 14, Selected False, Rank: 25.000
Column: 15, Selected False, Rank: 47.000
Column: 16, Selected False, Rank: 65.000
Column: 17, Selected False, Rank: 73.000
Column: 18, Selected False, Rank: 93.000
Column: 19, Selected False, Rank: 111.000
Column: 20, Selected False, Rank: 131.000
Column: 21, Selected False, Rank: 41.000
Column: 22, Selected False, Rank: 113.000
Column: 23, Selected False, Rank: 86.000
Column: 24, Selected Fa

In [144]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_Safe_ALP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [123, 124, 126, 129, 136]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,1,0,62.21
134,0,0,54.45
107,0,0,71.83
78,0,0,52.98
128,0,0,53.18
65,0,0,52.80


In [112]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,RELP - Buddhism,RELP - Christianity,RELP - Islam,RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Not married
0,0.420441,-0.691813,0.851753,-0.735348,0.490631
1,0.420441,-0.691813,0.851753,-0.735348,0.490631


# 4.4) Safe ALP (90% Accurate)

## Strongest Determinants
- Islam (0.851753)
- Not married (0.490631)
- Buddhism (0.420441)

## Negative determinants
- Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation (-0.735348)
- Christianity (-0.691813)

Interesting how religion is so important in determining whether an electorate is a safe ALP seat or not.

# 4.5) Safe LNP

In [113]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [114]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Safe_LNP']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Safe_LNP']

In [115]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.725
Accuracy score on testing set:  0.8064516129032258


In [116]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.8064516129032258
Accuracy on test set using 2 features:  0.8064516129032258
Accuracy on test set using 3 features:  0.8064516129032258
Accuracy on test set using 4 features:  0.8064516129032258
Accuracy on test set using 5 features:  0.8064516129032258
Accuracy on test set using 6 features:  0.8064516129032258
Accuracy on test set using 7 features:  0.8064516129032258
Accuracy on test set using 8 features:  0.8064516129032258
Accuracy on test set using 9 features:  0.8064516129032258
Accuracy on test set using 10 features:  0.8064516129032258
Accuracy on test set using 11 features:  0.8064516129032258
Accuracy on test set using 12 features:  0.8064516129032258
Accuracy on test set using 13 features:  0.8064516129032258
Accuracy on test set using 14 features:  0.8064516129032258
Accuracy on test set using 15 features:  0.8064516129032258
Accuracy on test set using 16 features:  0.8064516129032258
Accuracy on test set using 17 features:  0.806451

In [122]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=10, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.8064516129032258


In [123]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 30.000
Column: 1, Selected False, Rank: 81.000
Column: 2, Selected False, Rank: 39.000
Column: 3, Selected False, Rank: 76.000
Column: 4, Selected False, Rank: 7.000
Column: 5, Selected True, Rank: 1.000
Column: 6, Selected True, Rank: 1.000
Column: 7, Selected False, Rank: 18.000
Column: 8, Selected False, Rank: 59.000
Column: 9, Selected False, Rank: 72.000
Column: 10, Selected False, Rank: 35.000
Column: 11, Selected False, Rank: 25.000
Column: 12, Selected False, Rank: 22.000
Column: 13, Selected False, Rank: 13.000
Column: 14, Selected False, Rank: 21.000
Column: 15, Selected False, Rank: 28.000
Column: 16, Selected False, Rank: 46.000
Column: 17, Selected False, Rank: 65.000
Column: 18, Selected False, Rank: 88.000
Column: 19, Selected False, Rank: 104.000
Column: 20, Selected False, Rank: 121.000
Column: 21, Selected False, Rank: 6.000
Column: 22, Selected False, Rank: 54.000
Column: 23, Selected True, Rank: 1.000
Column: 24, Selected False, Rank

In [124]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_Safe_LNP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [5, 6, 23, 25, 51, 86, 90, 124, 126, 136]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,0,0,62.21
134,0,0,54.45
107,0,0,71.83
78,0,0,52.98
128,0,0,53.18
65,0,0,52.80


In [125]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,AGE5P - 25-29 years,AGE5P - 30-34 years,HEAP - Bachelor Degree Level,HEAP - Certificate III & IV Level,"INDP - Agriculture, Forestry and Fishing",MSTP - Never married,MSTP - Married,RELP - Christianity,RELP - Islam,MDCP - Not married
0,-0.380901,-0.375461,-0.525477,0.465158,0.384531,-0.738896,0.500072,1.403129,-0.560254,-0.657768
1,-0.380901,-0.375461,-0.525477,0.465158,0.384531,-0.738896,0.500072,1.403129,-0.560254,-0.657768


# 4.5) Safe LNP Interpretation (81% Accuracy)

## Main determinants
- Christianity (1.438735)
- Married (0.500072)

## Negative determinants
- Never married (-0.772616)
- Not married (-0.675473)
- Islam (-0.571603)
- Bachelor Degree Level (-0.554918)

The change of relatively higher percentages of those with their highest level of education being a Bachelors degree being a positive for Fairly safe LNP seat to a very strong negative for Safe LNP seat is interesting. 

# 4.6) Other Model (strength)

In [126]:
train, test = train_test_split(adjusted_master, test_size=0.2, random_state=142)
print(train.shape)
print(test.shape)

(120, 152)
(31, 152)


In [127]:
X_train = train.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_train = train['is_Strength_Other']
X_test = test.drop(['index', 'LGA','PartyNm', 'PreferencePercent', 'Strength', 'is_ALP', 'is_LNP', 'is_Other', 'is_Marginal',
                     'is_Fairly_Safe_ALP', 'is_Fairly_Safe_LNP', 'is_Safe_ALP', 'is_Safe_LNP', 'is_Strength_Other'], axis=1)
y_test = test['is_Strength_Other']

In [128]:
model = LogisticRegression()
model.fit(X_train, y_train)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)
print("Accuracy score on training set: ", accuracy_score(y_train, y_hat_train))
print("Accuracy score on testing set: ", accuracy_score(y_test, y_hat_test))

Accuracy score on training set:  0.975
Accuracy score on testing set:  0.967741935483871


In [129]:
acc_scores = []
for i in range(1,140):
    clf = LogisticRegression()
    rfe = RFE(estimator=clf, n_features_to_select=i)
    
    rfe.fit(X_train, y_train)
    
    y_pred = rfe.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    
    print("Accuracy on test set using", i, "features: ", acc_score)
    
    acc_scores.append(acc_score)

best = acc_scores[0]
best_number_of_columns = 0;
for i in range(len(acc_scores)):
    if acc_scores[i] > best:
        best = acc_scores[i]
        best_number_of_columns = i
        
print(best_number_of_columns + 1, 'columns provide the highest accuracy at: %.2f' % (best*100), "%")

Accuracy on test set using 1 features:  0.967741935483871
Accuracy on test set using 2 features:  0.967741935483871
Accuracy on test set using 3 features:  0.967741935483871
Accuracy on test set using 4 features:  0.967741935483871
Accuracy on test set using 5 features:  0.967741935483871
Accuracy on test set using 6 features:  0.967741935483871
Accuracy on test set using 7 features:  0.967741935483871
Accuracy on test set using 8 features:  0.967741935483871
Accuracy on test set using 9 features:  0.967741935483871
Accuracy on test set using 10 features:  0.967741935483871
Accuracy on test set using 11 features:  0.967741935483871
Accuracy on test set using 12 features:  0.967741935483871
Accuracy on test set using 13 features:  0.967741935483871
Accuracy on test set using 14 features:  0.967741935483871
Accuracy on test set using 15 features:  0.967741935483871
Accuracy on test set using 16 features:  0.967741935483871
Accuracy on test set using 17 features:  0.967741935483871
Accura

In [134]:
lr_model = LogisticRegression()
rfe = RFE(estimator=lr_model, n_features_to_select=5, step=1)
rfe.fit(X_train, y_train)
y_test_hat = rfe.predict(X_test)
print("accuracy score on test set: ", accuracy_score(y_test, y_test_hat))

accuracy score on test set:  0.967741935483871


In [135]:
relevant_cols = []
for i in range(X_train.shape[1]):
    print('Column: %d, Selected %s, Rank: %.3f' % (i, rfe.support_[i], rfe.ranking_[i]))
    if rfe.support_[i] == True:
        relevant_cols.append(i)
        
print(relevant_cols)

Column: 0, Selected False, Rank: 63.000
Column: 1, Selected False, Rank: 72.000
Column: 2, Selected False, Rank: 65.000
Column: 3, Selected False, Rank: 56.000
Column: 4, Selected False, Rank: 29.000
Column: 5, Selected False, Rank: 44.000
Column: 6, Selected False, Rank: 45.000
Column: 7, Selected False, Rank: 97.000
Column: 8, Selected False, Rank: 51.000
Column: 9, Selected False, Rank: 61.000
Column: 10, Selected False, Rank: 59.000
Column: 11, Selected False, Rank: 50.000
Column: 12, Selected False, Rank: 91.000
Column: 13, Selected False, Rank: 101.000
Column: 14, Selected False, Rank: 71.000
Column: 15, Selected False, Rank: 105.000
Column: 16, Selected False, Rank: 129.000
Column: 17, Selected False, Rank: 109.000
Column: 18, Selected False, Rank: 123.000
Column: 19, Selected False, Rank: 130.000
Column: 20, Selected False, Rank: 132.000
Column: 21, Selected False, Rank: 14.000
Column: 22, Selected False, Rank: 70.000
Column: 23, Selected False, Rank: 3.000
Column: 24, Selected

In [136]:
print("Columns that provide the best are:", relevant_cols)
result = pd.DataFrame(test['is_Safe_LNP'])
result.columns = ["Actual"]
result.insert(1, "Predicted", rfe.predict(X_test))
result.insert(2, "Pref Percent", test['PreferencePercent'].values)
display(result)

Columns that provide the best are: [39, 90, 92, 129, 134]


,Actual,Predicted,Pref Percent
3,0,0,56.26
70,0,0,58.36
92,0,0,53.28
124,0,0,57.52
56,0,0,62.21
134,0,0,54.45
107,0,0,71.83
78,0,0,52.98
128,0,0,53.18
65,0,0,52.80


In [137]:
coefficient_DF = pd.DataFrame(list(map(np.ravel, rfe.estimator_.coef_)), columns=X_train.columns[rfe.support_], index=rfe.estimator_.classes_)
coefficient_DF

,"INCP - $3,000 or more ($156,000 or more)",MSTP - Married,"RLHP - Husband, Wife or Partner in a registered marriage",RELP - Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation,MDCP - Married in a registered marriage
0,0.061699,-0.072724,-0.077377,0.120757,-0.07698
1,0.061699,-0.072724,-0.077377,0.120757,-0.07698


# 4.6 Other Interpretation (97% Accuracy)

## Strongest determinants
- Secular Beliefs and Other Spiritual Beliefs and No Religious Affiliation (0.120757)
- 3,000 or more( 156,000 or more) (0.061699)

## Negative
- Husband, Wife or Partner in a registered marriage (-0.077377)
- Married in a registered marriage (-0.07698)
- Married (-0.072724)

Note, like predicting other in the 3 model for other, the coefficients are all very low/ weak hence even though we supposedly have a 97% accuracy, this is due to having very few data points in this category. Would not use this evidence that other is well predicted for reasons as stated in the 3- model of the other category, they are very similar.  